**Preprocessing**

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("ServiceNowQuestions_clean.xlsx")

In [3]:
df.drop('Tags', inplace=True, axis=1)
df.drop('index', inplace=True, axis=1)
df.drop('Title', inplace=True, axis=1)
df

,level_0,Question,Answer
0,12935,"Hi All,We have an input where we will get a va...",So you need to write this in server-side scrip...
1,12936,I am trying to find the most recent date in a ...,"Hi Wade,I was going to try to step through tro..."
2,12937,"Hello,I am having problems getting my addQuery...",Please see below for how to troubleshoot and s...
3,12938,"Hi everyone, I'm looking to pick some brains a...",I have come across 2 solutions to address this...
4,12939,hii have one email notification in which i wan...,"Hi User,Please follow below link or stepdhttps..."
...,...,...,...
5739,18688,Is there a straight forward way to replicate t...,"Hi David [davidsizu ],This is a custom impleme..."
5740,18689,"Hi,How do I go about making the UI Action butt...",Try something like: gs.getUser().isMemeberOf(c...
5741,18690,I want to apply validations to email . AS i Ca...,This is a system limitation. Field validations...
5742,18691,"Hello Experts,I have gotten a request from a c...","Its a bit much code to include here, so I have..."


In [4]:
df = df.astype({"Question": str})

In [5]:
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords

In [6]:
def clean_sentence(s, stopWords=False):
    s = s.lower().strip()
    #s = re.sub(r'[^a-z0-9\s]',' ',s)
    if stopWords:
        s = remove_stopwords(s)
    return s

def get_cleaned(df, stopWords=False):
    sents = df[["Question"]]
    cleaned_sentences = []
    for index, row in df.iterrows():
        cleaned = clean_sentence(row["Question"], stopWords)
        cleaned_sentences.append(cleaned)
    return cleaned_sentences

cleaned_s_with_not_stopword = get_cleaned(df, stopWords = True)

print(cleaned_s_with_not_stopword[0])

print("\n")
cleaned_s_with_stopword = get_cleaned(df, stopWords = False)
    
print(cleaned_s_with_stopword[0])

hi all,we input value short_description (for example lets say: 1 50 numbers short_description)i want value 1 30 characters.how trim short_description value 30 charactersregards,reddy


hi all,we have an input where we will get a value as short_description (for example lets say: 1 to 50 numbers in short_description)i want the value which is after 1 to 30 characters.how to trim the short_description value more than 30 charactersregards,reddy


In [7]:
df["Question"][0]

'Hi All,We have an input where we will get a value as short_description (For example lets say: 1 to 50 numbers in short_description)I want the value which is after 1 to 30 characters.How to trim the short_description value more than 30 charactersRegards,Reddy'

In [8]:

print(cleaned_s_with_not_stopword[1])

print("\n")

    
print(cleaned_s_with_stopword[1])

trying recent date function query attempt add 7 days reassign original field.here ui action tried, takes current date adds 7 days. sure why, query failure tell wrong.any help appreciated. thanks advance!var copyhours = false;try { var ts = new gliderecord('u_time_sheet'); ts.u_state = 'pending'; wsodate = getnextdate(); ts.u_weeks_start_on = wsodate.adddays(7); ts.u_user = current.u_user; var newid = ts.insert(); var te = new gliderecord('u_time_entry'); te.addquery('u_time_sheet', current.sys_id); te.query(); while(te.next()) { var tenew = new gliderecord('u_time_entry'); tenew.initialize(); tenew.u_time_sheet = newid; tenew.user = te.user; tenew.task = te.task; tenew.u_description = te.u_description; tenew.category = te.category; tenew.u_task_type = te.u_task_type; tenew.u_comments = te.u_comments; tenew.week_starts_on = te.week_starts_on; tenew.setworkflow(false); tenew.insert(); } gs.clearmessages(); gs.adderrormessage('a new timesheet created, verify week starts date update submis

In [9]:
df["Question"][1]

'I am trying to find the most recent date in a function query in an attempt to add 7 days and reassign it to the original field.Here is the UI Action that I have tried, but it still only takes the current date and adds 7 days.  Not sure why, at least a query failure would tell me if I was wrong.Any help would be appreciated.  Thanks in advance!var copyHours = false;try {  var ts = new GlideRecord(\'u_time_sheet\');  ts.u_state = \'Pending\';  wsodate = getnextdate();  ts.u_weeks_start_on = wsodate.addDays(7);  ts.u_user = current.u_user;  var newID = ts.insert();  var te = new GlideRecord(\'u_time_entry\');  te.addQuery(\'u_time_sheet\', current.sys_id);  te.query();  while(te.next()) {  var teNew = new GlideRecord(\'u_time_entry\');  teNew.initialize();  teNew.u_time_sheet = newID;  teNew.user = te.user;  teNew.task = te.task;  teNew.u_description = te.u_description;  teNew.category = te.category;  teNew.u_task_type = te.u_task_type;  teNew.u_comments = te.u_comments;  teNew.week_star

In [ ]:
from bert_serving.client import BertClient
bc = BertClient()
